In [70]:
from bs4 import BeautifulSoup
import requests
import pymongo
import time
import random
import re
import pdb

## Ulta Scraping
### Exploration of the soup 

In [14]:
user_agent =  'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
html_page = requests.get('https://www.ulta.com/hair-styling-products?N=26xf&Nrpp=96&Ns=product.bestseller%7C1', headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="en">
<head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=0" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<meta content="0iJgVZRwJEqcwh-DfHO3TnKtFasUWdJmJbVBo2j8eg0" name="google-site-verification"/>
<title>Styling Products | Ulta Beauty</title>
<meta content="Styling Products | Ulta Beauty" name="title"/>
<meta content="Oracle Commerce reference application." name="description"/>
<link href="https://www.ulta.com/hair-styling-products?N=26xf" rel="canonical"/>
<link href="/ui/static/css/vendor.1566326797713.css" rel="stylesheet"/><link href="/ui/static/css/mhp.1566326797713.css" rel="stylesheet"/><link href="//statica.ultainc.com/static/css/combineCommonHeader.css" rel="stylesheet" type="text/css">
<link href="//statica.ultainc.com/static/css/combineHeader.css" rel="stylesheet" type="text/css">
<scri

In [57]:
xdesc = soup.find_all("p", class_='prod-desc')[95].text
xtitle = soup.find_all("h4", class_="prod-title")[0].text

In [58]:
xdesc

'\n\n\t\t\t\tBed Head Foxy Curls Extreme Curl Mousse\n'

In [53]:
xlist = []
xlist.append(xtitle + xdesc)
xlist

['\n\n\t\t\t\tTigi\n\n\n\t\t\t\tBed Head Masterpiece Shine Hairspray\n']

In [54]:
s = re.sub(r"\n", "", xlist[0])
s = re.sub(r"\t", "", s)
xlist.append(s)
xlist

['\n\n\t\t\t\tTigi\n\n\n\t\t\t\tBed Head Masterpiece Shine Hairspray\n',
 'TigiBed Head Masterpiece Shine Hairspray']

In [ ]:
# find how to increment pages
#page 1
https://www.ulta.com/hair-styling-products?N=26xf&Nrpp=96&Ns=product.bestseller%7C1
#page 2
https://www.ulta.com/hair-styling-products?N=26xf&No=96&Nrpp=96&Ns=product.bestseller%7C1
#page 3
https://www.ulta.com/hair-styling-products?N=26xf&No=192&Nrpp=96&Ns=product.bestseller%7C1
https://www.ulta.com/hair-styling-products?N=26xf&No=1056&Nrpp=96&Ns=product.bestseller%7C1    

In [9]:
list(range(0,1056+1,96))

[0, 96, 192, 288, 384, 480, 576, 672, 768, 864, 960, 1056]

In [ ]:
https://www.ulta.com/hair-shampoo-conditioner?N=27ih
https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No=96&Nrpp=96
https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No=192&Nrpp=96
https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No=1440&Nrpp=96    

In [61]:
list(range(0,1440+1,96))

[0,
 96,
 192,
 288,
 384,
 480,
 576,
 672,
 768,
 864,
 960,
 1056,
 1152,
 1248,
 1344,
 1440]

### Scrape funcitonality

In [59]:
# Scrape the Ulta Styling products section

ulta_styling = []
for i in list(range(0,1056+1,96)):
    url = f'https://www.ulta.com/hair-styling-products?N=26xf&No={i}&Nrpp=96&Ns=product.bestseller%7C1'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    
    
    for x in range(0,96):
        try:
            xdesc = soup.find_all("p", class_='prod-desc')[x].text
            xtitle = soup.find_all("h4", class_="prod-title")[x].text
            
            # do some cleaning
            s = re.sub(r"\n", "", xdesc)
            s = re.sub(r"\t", "", s)
            new_xdesc = s

            s2 = re.sub(r"\n", "", xtitle)
            s2 = re.sub(r"\t", "", s2)
            new_xtitle = s2


            ulta_styling.append(new_xtitle + " " + new_xdesc)
        
        except:    
            continue
        

In [62]:
ulta_styling[0:20]

['Tigi Bed Head Masterpiece Shine Hairspray',
 'Drybar The Blowout Brigade Set',
 'Kenra Professional Volume Spray 25',
 'Tigi Bed Head After-Party',
 'Sexy Hair Big Sexy Hair Spray & Play Volumizing Hairspray',
 'Sebastian Shaper Plus Extra Hold Hairspray',
 'Kenra Professional Platinum Blow-Dry Spray',
 "Bumble and bumble Bb.Hairdresser's Invisible Oil Heat/UV Protective Primer",
 'Sexy Hair Big Sexy Hair Spray & Play Harder Firm Volumizing Hairspray',
 'Living Proof Online Only No Frizz Favorites Jumbo Bundle',
 'Tigi Bed Head Hard Head Hairspray',
 'Sexy Hair Big Sexy Hair Root Pump Plus Humidity Resistant Volumizing Spray Mousse',
 'DevaCurl Ultra Defining Gel Strong Hold No-Crunch Styler',
 'Curlsmith Curl Conditioning Oil-In-Cream',
 'Tigi Catwalk Curls Rock Amplifier',
 'Curlsmith Curl Quenching Conditioning Wash',
 'Redken Control Addict 28 Extra High-Hold Hairspray',
 'AG Hair Curl Re:Coil Curl Activator',
 'Ouidad Advanced Climate Control Heat & Humidity Gel',
 'Sebastian Sh

In [66]:
len(ulta_styling)

1145

In [ ]:
# Scrape the Ulta Shampoo and conditioner products section

ulta_cleansers = []
for i in list(range(0,1440+1,96)):
    url = f'https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No={i}&Nrpp=9'
    html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
    soup = BeautifulSoup(html_page.content, 'html.parser') #Pass the page contents to beautiful soup for parsing
    
    
    for x in range(0,96):
        try:
            xdesc = soup.find_all("p", class_='prod-desc')[x].text
            xtitle = soup.find_all("h4", class_="prod-title")[x].text
            
            # do some cleaning
            s = re.sub(r"\n", "", xdesc)
            s = re.sub(r"\t", "", s)
            new_xdesc = s

            s2 = re.sub(r"\n", "", xtitle)
            s2 = re.sub(r"\t", "", s2)
            new_xtitle = s2


            ulta_cleansers.append(new_xtitle + " " + new_xdesc)
        
        except:  
            pdb.set_trace()
            continue
        

> <ipython-input-71-16cd3c582d56>(29)<module>()
-> continue
(Pdb) new_xtitle
'Redken'
(Pdb) new_xdesc
'Color Extend Magnetics Shampoo'
(Pdb) ulta_cleansers.append(new_xtitle + " " + new_xdesc)
(Pdb) ulta_cleansers
['Redken All Soft Conditioner', 'Paul Mitchell Tea Tree Special Shampoo', 'Redken All Soft Shampoo', 'Matrix Biolage Colorlast Conditioner', 'Matrix Biolage Colorlast Shampoo', 'Paul Mitchell Tea Tree Lavender Mint Moisturizing Conditioner', 'Paul Mitchell Tea Tree Lavender Mint Moisturizing Shampoo', 'Matrix Biolage Ultra Hydrasource Conditioner', 'Redken Color Extend Magnetics Shampoo', 'Redken Color Extend Magnetics Shampoo']
(Pdb) soup.find_all("p", class_='prod-desc')[x].text
*** IndexError: list index out of range
(Pdb) soup.find_all("p", class_='prod-desc')
[<p class="prod-desc">
<a href="/all-soft-conditioner?productId=xlsImpprod15511073">
				All Soft Conditioner</a>
</p>, <p class="prod-desc">
<a href="/tea-tree-special-shampoo?productId=xlsImpprod10271369">
				Tea

In [68]:
ulta_cleansers

['Redken All Soft Conditioner',
 'Paul Mitchell Tea Tree Special Shampoo',
 'Redken All Soft Shampoo',
 'Matrix Biolage Colorlast Conditioner',
 'Matrix Biolage Colorlast Shampoo',
 'Paul Mitchell Tea Tree Lavender Mint Moisturizing Conditioner',
 'Paul Mitchell Tea Tree Lavender Mint Moisturizing Shampoo',
 'Matrix Biolage Ultra Hydrasource Conditioner',
 'Redken Color Extend Magnetics Shampoo',
 'Hempz Original Herbal Conditioner for Damaged & Color Treated Hair',
 "It's A 10 Silk Express Miracle Silk Shampoo",
 'Matrix Biolage Advanced Fiberstrong Shampoo for Fragile Hair',
 'Pureology Strength Cure Shampoo',
 'Nioxin Scalp Therapy Conditioner System 2 For Fine Hair With Progressed Thinning',
 'Kenra Professional Platinum Luxe Shine Conditioner',
 'Hempz Original Herbal Shampoo for Damaged & Color Treated Hair',
 'Matrix Biolage Advanced Full Density Conditioner for Thin Hair',
 'DevaCurl One Condition Decadence Ultra Moisturizing Milk Conditioner',
 'Matrix Total Results Keep Me Vi

In [69]:
len(ulta_cleansers)

144

In [ ]:
url = f'https://www.ulta.com/hair-shampoo-conditioner?N=27ih&No={0}&Nrpp=9'
html_page = requests.get(url, headers = {'User-Agent': user_agent} ) #Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser')
xdesc = soup.find_all("p", class_='prod-desc')[9].text
xtitle = soup.find_all("h4", class_="prod-title")[9].text
print((xdesc))
print((xtitle))